In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import numpy
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [3]:

train = pd.read_csv('../train.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])
test = pd.read_csv('test.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])
# train = normailize_df(train, True)
# test = normailize_df(test, False)
train.drop(columns=["direccion", "lat", "lng"], inplace=True)
test.drop(columns=["direccion", "lat", "lng"], inplace=True)
train = train[~train.tipodepropiedad.isnull()]
df_all = train.append(test)
df_all['fecha'] = df.fecha.values.astype(np.int64)
test[test.tipodepropiedad.isnull()]

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
# df_all[:train] = np.log(df_train.SalePrice)
df_all['preciomt2'] = df_all['precio'] / (df_all['metroscubiertos'] + df_all['metrostotales'])
df_all['logpreciomt2'] = np.log1p(df_all.preciomt2)
df_all['logprecio'] = np.log1p(df_all.precio)

corr = df_all.corr().abs()
corr.precio[corr.precio >= 0.5].sort_values(ascending=False)

In [ ]:
def null_values(df):
    
    sum_null = df.isnull().sum()
    total = df.isnull().count()
    percent_nullvalues = 100* sum_null / total 
    df_null = pd.DataFrame()
    df_null['Total'] = total
    df_null['Null_Count'] = sum_null
    df_null['Percent'] = round(percent_nullvalues,2)
    df_null = df_null.sort_values(by='Null_Count',ascending = False)
    df_null = df_null[df_null.Null_Count > 0]
    
    return(df_null)
null_values(train)

In [ ]:
null_values(train[train['metrostotales'].isnull()])

In [ ]:
null_values(train[(train['metroscubiertos'].isnull())])
#los que no tienen metros cubiertos mayormente tampoco tienen baños declarados. Podemos asumir
#si no hay metros cubiertos entonces --> no hay banos 
# train[train['metroscubiertos'].isnull()].tipodepropiedad.value_counts()

In [ ]:
# Analisemos cuidadosamente los terrenos primero
null_values(train[(train['metroscubiertos'].isnull()) & (train['tipodepropiedad'] == 'Terreno')])

In [ ]:
corr = df_all[df_all['tipodepropiedad'] == 'Terreno'].corr()
corr['precio']

In [ ]:
condition = (df_all['metroscubiertos'].isnull()) & (df_all['tipodepropiedad'] == 'Terreno')
df_all.loc[condition, 'garages'].hist(bins=30)

In [ ]:
condition = (df_all['metroscubiertos'].isnull()) & (df_all['tipodepropiedad'] == 'Terreno Comercial')
df_all.loc[condition, 'antiguedad'].hist(bins=30)

In [ ]:
# Vemos que los que tienen valores nulos en metros cubiertos tambien tienen banos, habitaciones, garages y antiguedad. Consideramos todos estos 
# como 0 entonces.
condition = (df_all['metroscubiertos'].isnull()) & (df_all['tipodepropiedad'] == 'Terreno')
df_all.loc[condition, 'banos'] = 0
df_all.loc[condition, 'habitaciones'] = 0
df_all.loc[condition, 'metroscubiertos'] = 0
df_all.loc[condition, 'garages'] = 0
df_all.loc[condition, 'antiguedad'] = 0

In [ ]:
condition = (df_all['metrostotales'].isnull()) & (df_all['tipodepropiedad'] == 'Terreno')

In [ ]:
# En este caso vemos que hay distribucion en las antiguedades distinta de 0. Esto podria tenerlo en cuenta 
# el algoritmo que usemos. Por lo que solo llenaremos los NULOS con 0 y no todos.
df_all[condition].habitaciones.hist(bins=30)

In [ ]:
#normalizemos solo los nulos
condition = (df_all['metrostotales'].isnull()) & (df_all['tipodepropiedad'] == 'Terreno')

for col in ['antiguedad', 'habitaciones', 'banos', 'garages', 'metrostotales', 'gimnasio']:
    df_all.loc[condition, col] = df_all[condition][col].fillna(0)
    

In [ ]:
corr = df_all[df_all['tipodepropiedad'] == 'Terreno'].corr()
corr['precio']

In [ ]:
df_all[df_all.metrostotales.isnull()].tipodepropiedad.value_counts()

In [ ]:
#Veamos ahora apartamentos que tengan completado metrostotales y metroscubiertos. 80% poseen el mismo valor en ambos.
notnull = (df_all['tipodepropiedad'] == 'Apartamento') & (~df_all['metrostotales'].isnull()) & (~df_all['metroscubiertos'].isnull())
desigualmt2notnull = (df_all['tipodepropiedad'] == 'Apartamento') & (df_all['metrostotales'] != df_all['metroscubiertos']) & (~df_all['metrostotales'].isnull()) & (~df_all['metroscubiertos'].isnull())
igualmt2notnull = (df_all['tipodepropiedad'] == 'Apartamento') & (df_all['metrostotales'] == df_all['metroscubiertos']) & (~df_all['metrostotales'].isnull()) & (~df_all['metroscubiertos'].isnull())
null = (df_all['tipodepropiedad'] == 'Apartamento') & (df_all['metrostotales'].isnull() | df_all['metroscubiertos'].isnull())

# import pandas.rpy.common as com
import seaborn as sns


# load the R package ISLR
# infert = com.importr("ISLR")

# load the Auto dataset
# auto_df = com.load_data('Auto')

# calculate the correlation matrix
corr = df_all[df_all['tipodepropiedad'] == 'Apartamento'].corr()
corr['precio']
# plot the heatmap
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap="YlGnBu")
# df_all[desigualmt2notnull].shape[0] / df_all[notnull].shape[0]

In [ ]:
#Veamos ahora apartamentos que tengan completado metrostotales y metroscubiertos.
corr2 = df_all[igualmt2notnull].corr()
(corr - corr2) > 0
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap="YlGnBu")
# df_all[igualmt2notnull].shape[0] / df_all[notnull].shape[0]

In [ ]:
df_all[df_all.tipodepropiedad == 'Casa'].isnull().sum()

In [ ]:
null_values(df_all[(df_all.tipodepropiedad == 'Casa')])

In [ ]:
def resolverCampo(tipodepropiedad, campo, test):
    pred_train = df_all[(df_all.tipodepropiedad == tipodepropiedad) & (~df_all[campo].isnull())] 
    pred_test = df_all[(df_all.tipodepropiedad == tipodepropiedad) & (df_all[campo].isnull())]

    every_column_except_y= [col for col in pred_train.columns if col not in [campo, 'id', 'ciudad', 'fecha', 'provincia', 'tipodepropiedad', 'precio', 'descripcion', 'titulo']]
    
    if(pred_train[every_column_except_y].shape[0] <= 1): return 0
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                    max_depth = 5, alpha = 10, n_estimators = 400, n_jobs=40)
    
    if(test): 
        X_train, X_test, y_train, y_test = train_test_split(pred_train[every_column_except_y], pred_train[campo], test_size=0.2, random_state=123)
        xg_reg.fit(X_train,y_train)
        pred= xg_reg.predict(X_test)     
        return np.sqrt(mean_squared_error(y_test, pred))
    else:    
        xg_reg.fit(pred_train[every_column_except_y],pred_train[campo])
        pred= xg_reg.predict(pred_test[every_column_except_y])
        return pred

In [ ]:
df_all['metrostotales'] = np.log1p(df_all['metrostotales'])
df_all['metroscubiertos'] = np.log1p(df_all['metroscubiertos'])
df_all['metrostotales'] = df_all['metrostotales'].fillna(0)
df_all['metroscubiertos'] = df_all['metroscubiertos'].fillna(0)
# for el in ['Apartamento','Casa', 'Casa en condominio', 'Local Comercial', 'Oficina comercial', 'Quinta Vacacional', 'Bodega comercial', 'Terreno comercial', 'Villa', 'Rancho', 'Edificio', 'Casa uso de suelo', 'Inmuebles productivos urbanos', 'Local en centro comercial', 'Departamento Compartido', 'Otros', 'Nave industrial', 'Duplex', 'Huerta', 'Terreno industrial', 'Garage', 'Hospedaje']:

#     mtcubant = df_all.loc[(df_all.tipodepropiedad == el)].corr()['precio']['metroscubiertos']
#     mtcubtot = df_all.loc[(df_all.tipodepropiedad == el)].corr()['precio']['metrostotales']
    
#     df_all.loc[(df_all.tipodepropiedad == el) & (df_all['metroscubiertos'].isnull()), 'metroscubiertos'] = resolverCampo(el, 'metroscubiertos', False)
#     df_all.loc[(df_all.tipodepropiedad == el) & (df_all['metrostotales'].isnull()), 'metrostotales'] = resolverCampo(el, 'metrostotales', False)
#     mtcubdes = df_all.loc[(df_all.tipodepropiedad == el)].corr()['precio']['metroscubiertos']
#     mtcubdestot = df_all.loc[(df_all.tipodepropiedad == el)].corr()['precio']['metrostotales']

# df_all['metrostotales'] = np.expm1(df_all['metrostotales'])
# df_all['metroscubiertos'] = np.expm1(df_all['metroscubiertos'])
    
    

In [ ]:
df_all.loc[:, 'banos'] = df_all['banos'].fillna(0)
df_all.loc[:, 'habitaciones'] = df_all['habitaciones'].fillna(0)
df_all.loc[:, 'metroscubiertos'] = df_all['metroscubiertos'].fillna(0)
df_all.loc[:, 'garages'] = df_all['garages'].fillna(0)
df_all.loc[:, 'metrostotales'] = df_all['metrostotales'].fillna(0)

In [ ]:
null_values(df_all)

In [ ]:

import category_encoders as ce
#Labeling

cols = ['tipodepropiedad', 'provincia', 'ciudad'] 
         #, 'provincia', 'ciudad'
        
encoder = ce.BinaryEncoder(cols=cols)
df_all = encoder.fit_transform(df_all)

In [ ]:
# df_all[:train] = np.log(df_train.SalePrice)
df_all['metros'] = np.abs(df_all['metroscubiertos'] + df_all['metrostotales'])
df_all['preciomt2'] = df_all['precio'] / (df_all['metroscubiertos'] + df_all['metrostotales'])
df_all['logpreciomt2'] = np.log1p(df_all.preciomt2)

corr = df_all.corr().abs()
corr.precio[corr.precio >= 0.5].sort_values(ascending=False)

In [ ]:
from numpy.random import seed
from numpy.random import randn
from numpy import percentile
#
# calculate interquartile range
q25, q75 = percentile(df_all[:train.shape[0]].logpreciomt2, 25), percentile(df_all[:train.shape[0]].logpreciomt2, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
train = df_all[:train.shape[0]] 
test = df_all[train.shape[0]:] 
train = train[~((train.logpreciomt2 < lower) | (train.logpreciomt2 > upper) & (train.logpreciomt2 != 0))]
df_all = train.append(test)


# train.shape
# outliers = [x for x in df_all[:train.shape[0]].logpreciomt2 if x < lower or x > upper]
# print('Identified outliers: %d' % len(outliers))
# outliers
# outliers_removed = [x for x in data if x >= lower and x <= upper]
# print('Non-outlier observations: %d' % len(outliers_removed))


In [ ]:
plt.scatter(train.metroscubiertos, train.logpreciomt2, c = "blue", marker = "s")
plt.title("Metros vs logpreciomt2")
plt.xlabel("Metros")
plt.ylabel("logpreciomt2")
plt.show()

In [ ]:
from scipy.stats import skew

#log transform skewed numeric features:
numeric_feats = df_all.dtypes[(df_all.dtypes == "float64") | (df_all.dtypes == "int64")].index
skewed_feats = df_all[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = [col for col in skewed_feats.index if col not in ['id', 'precio', 'preciomt2']] 

In [ ]:
# final = test[['id']]
# final.loc[:, 'target'] = np.expm1(pred)
train.set_index('id').to_csv('../normalized_train.csv')
test.set_index('id').to_csv('../normalized_test.csv')
